In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
#from xgboost import XGBClassifier
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import f1_score,precision_score,confusion_matrix,recall_score,accuracy_score


In [2]:
df_fake= pd.read_csv('Fake.csv',  error_bad_lines=False, encoding= "latin-1")
df_true= pd.read_csv('True.csv', error_bad_lines=False, encoding= "latin-1")

b'Skipping line 11067: expected 4 fields, saw 8\n'


In [3]:
df_true['news_class'] = 1
df_fake['news_class'] = 0

In [4]:
df_true.head()

,title,text,subject,date,news_class
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [5]:
df_fake.head()

,title,text,subject,date,news_class
0,Donald Trump Sends Out Embarrassing New Year...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obamas Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [6]:
import re
def cleanResume(resumeText):#resumText perform several operations to clean and preprocess the text of resume.
    
    resumeText = re.sub(r'[\x1d\x1c\x1b]', '', resumeText)#remove special character
    resumeText = re.sub(r'\x19', ' ', resumeText) #to replace character \x19 with space ' '.
    resumeText = re.sub(r'\s+', ' ', resumeText).strip()#remove whitespace
    resumeText = re.sub('http\S+\s*', ' ', resumeText)  # remove URLs
    #resumeText = re.sub('#\S+', '', resumeText)  # remove hashtags
    #resumeText = re.sub('@\S+', '  ', resumeText)  # remove mentions
    resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resumeText)  # remove punctuations
    resumeText = re.sub(r'\d+', '', resumeText)#remove number 
    resumeText = re.sub('\s+', ' ', resumeText)  # remove extra whitespace
    return resumeText.lower()#returns clean "resumeText" and convert all text into lowercase.

In [7]:
df_true["text"]=df_true.text.apply(cleanResume)
df_fake["text"]=df_fake.text.apply(cleanResume)

In [8]:
from nltk.tokenize import word_tokenize
df_true["text"] = df_true.text.apply(word_tokenize)

In [9]:
df_fake["text"]= df_fake.text.apply(word_tokenize)

In [10]:
#removing Stop words 
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [11]:
df_true["text"] = df_true.text.apply(lambda x: [word for word in x if word.lower() not in stop_words])

In [12]:
df_fake["text"] = df_fake.text.apply(lambda x: [word for word in x if word.lower() not in stop_words])

In [13]:
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [14]:
lemmatizer = WordNetLemmatizer()

df_true["text"] = df_true.text.apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

In [15]:
df_fake["text"] = df_fake.text.apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

In [16]:
df_true['corpus_t'] = df_true['text'].apply(lambda x: ' '.join(x))
corpus_t = df_true['corpus_t'].tolist()  # Convert the corpus_t column to a list
df_fake['corpus_f'] = df_fake['text'].apply(lambda x: ' '.join(x))
corpus_f = df_fake['corpus_f'].tolist()  # Convert the corpus_f column to a list


In [17]:
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk import ngrams

def get_ngrams(corpus, n):
    ngrams_list = []
    for text in corpus:
        tokens = word_tokenize(text)  # Tokenize the text into words
        ngrams_list.extend(list(ngrams(tokens, n)))  # Generate n-grams
    return ngrams_list

def get_top_ngrams(corpus_t, corpus_f, n, top_k=100):
    # Get n-grams for corpus_t
    ngrams_t = Counter(get_ngrams(corpus_t, n))

    # Get n-grams for corpus_f
    ngrams_f = Counter(get_ngrams(corpus_f, n))

    # Get top k frequent common n-grams
    common_ngrams = set(ngrams_t.keys()) & set(ngrams_f.keys())
    top_common_ngrams = sorted(common_ngrams, key=lambda x: ngrams_t[x] + ngrams_f[x], reverse=True)[:top_k]

    # Get top k unique n-grams for corpus_t
    unique_ngrams_t = set(ngrams_t.keys()) - set(ngrams_f.keys())
    top_unique_ngrams_t = sorted(unique_ngrams_t, key=lambda x: ngrams_t[x], reverse=True)[:top_k]

    # Get top k unique n-grams for corpus_f
    unique_ngrams_f = set(ngrams_f.keys()) - set(ngrams_t.keys())
    top_unique_ngrams_f = sorted(unique_ngrams_f, key=lambda x: ngrams_f[x], reverse=True)[:top_k]

    return top_common_ngrams, top_unique_ngrams_t, top_unique_ngrams_f

In [18]:
# Unigrams
n =1 
top_common_unigrams, top_unique_unigrams_t, top_unique_unigrams_f = get_top_ngrams(corpus_t, corpus_f, n)

In [19]:
# Bigrams
n = 2
top_common_bigrams, top_unique_bigrams_t, top_unique_bigrams_f = get_top_ngrams(corpus_t, corpus_f, n)

In [21]:
# Trigrams
n = 3
top_common_trigrams, top_unique_trigrams_t, top_unique_trigrams_f = get_top_ngrams(corpus_t, corpus_f, n)


In [23]:
def remove_ngrams(corpus, ngrams_list):
    for ngram in ngrams_list:
        while ngram in corpus:
            corpus.remove(ngram)

In [24]:
remove_ngrams(corpus_t, top_common_unigrams)
remove_ngrams(corpus_f, top_common_unigrams)
remove_ngrams(corpus_t, top_common_bigrams)
remove_ngrams(corpus_f, top_common_bigrams)
remove_ngrams(corpus_t, top_common_trigrams)
remove_ngrams(corpus_f, top_common_trigrams)

In [25]:
custom_stopwords=[
'according', 'administration','also', 'america', 'american', 'americans', 'another', 'back', 'bill', 'black', 'called', 'campaign', 'clinton', 'could', 'country', 'day', 'department', 'donald', 'election', 'even', 'every', 'fact', 'first', 'former', 'fox', 'get', 'go', 'going', 'good', 'government', 'group', 'hillary', 'house', 'Image', 'it', 'know', 'last', 'law', 'like', 'made', 'make', 'may', 'media', 'much', 'national', 'never', 'new', 'news', 'man', 'many', 'obama', 'office', 'one', 'party', 'people', 'police', 'political', 'president', 'presidential', 'public', 'really', 'republican', 'republicans', 'right', 'said', 'say', 'says', 'see', 'show', 'since', 'state', 'states', 'still', 'support', 'take', 'think', 'time', 'told', 'trump', 'two', 'united', 'us', 'via', 'video', 'vote', 'white', 'women', 'world', 'would', 'year', 'years', 'want'
]

In [26]:
df_true["text"] = df_true.text.apply(lambda x: [word for word in x if word.lower() not in custom_stopwords])
df_fake["text"] = df_fake.text.apply(lambda x: [word for word in x if word.lower() not in custom_stopwords])


In [27]:
corpus_t = df_true['text'].apply(lambda x: ' '.join(x)).tolist()
corpus_f = df_fake['text'].apply(lambda x: ' '.join(x)).tolist()


In [28]:
corpus = corpus_t + corpus_f

In [29]:
labels = [0] * len(corpus_f) + [1] * len(corpus_t)

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=1000)
tfidf_matrix = vectorizer.fit_transform(corpus)
tfidf_array = tfidf_matrix.toarray()
feature_names = vectorizer.get_feature_names()


In [32]:
#Random Forest Classifier
rf = RandomForestClassifier()
x_train, x_test, y_train, y_test = train_test_split(tfidf_array, labels, test_size=0.2, random_state=42)
rf.fit(x_train,y_train)
y_pred_rf = rf.predict(x_test)


In [33]:
### Create a pickle file using serialization 
import pickle
pickle_out = open("rf.pkl","wb")
pickle.dump(rf,pickle_out)
pickle_out.close()

In [39]:
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

In [34]:
import streamlit as st

In [35]:
pickle_in = open("rf.pkl","rb")
rf = pickle.load(pickle_in)

In [36]:
import joblib

In [37]:
#Load the trained model
model = joblib.load('rf.pkl')

In [38]:
# Define the Streamlit App
def main():
    # Add title and description
    st.title("Text Classification Demo")
    st.write("Enter the text and click on the 'Predict' button in order to classify if it is Fake or True.")

    # Create a text input box for user input
    text_input = st.text_area("Enter Text","")

    # Add a predict button
    if st.button("Predict"):
        # Preprocess the input text
        preprocessed_text = preprocessed_text(text_input)
        # Make a prediction using the loaded model
        prediction = model.predict([preprocessed_text])[0]
        # Map prediction to 'Fake or True'
        prediction_label = "Fake" if prediction == 0 else "True"
        # Display the prediction
        st.subheader("Prediction")
        st.write(prediction_label)

# Run the app
if __name__ == '__main__':
    main()

2023-07-08 14:10:16.935 
  command:

    streamlit run C:\Users\Dell\anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
